In [2]:
from pianoq.misc.misc import run_in_thread
from pianoq.misc.mplc_writeup_imports import * 
from pianoq.misc.mplc_lab_imports import * 

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_10_08_find_z10\finer'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

# Finding dz10

In [ ]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1 

dz10s = np.array([4, 8, 10, 12]) * 1e-3

for dz10 in dz10s:
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    masks_path =    fr'{dir_path}\{timestamp}_U1U_dz10={dz10}.masks'
    mplc_res_path = fr'{dir_path}\{timestamp}_U1U_dz10={dz10}.mplcsim'
    mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path, dead_middle_zone=0, last_plane_extra_dist=dz10)
    mplc_sim.res.saveto(mplc_res_path, smaller=True)

In [3]:
import re 
masks_paths = glob.glob(rf'{dir_path}\*.masks')
all_ds = np.array([re.findall('.*dz10=(.*).masks', path)[0] for path in masks_paths]).astype(float)

In [5]:
mplc = MPLCDevice()

backlash = 0
wait_after_move = 0.2
mxs, mys = get_signal_motors(backlash=backlash, wait_after_move=wait_after_move)

integration_time = 1
coin_window = 2e-9
tt = get_timetagger(coin_window=coin_window, integration_time=integration_time)

Got Zaber motors
God timetagger


In [4]:
import re 
masks_paths = glob.glob(rf'G:\My Drive\Projects\MPLC\results\lab\2024_10_08_find_z10\finer\*.masks')
all_ds = np.array([re.findall('.*dz10=(.*).masks', path)[0] for path in masks_paths]).astype(float)

['G:\\My Drive\\Projects\\MPLC\\results\\lab\\2024_10_08_find_z10\\finer\\2024_10_09_11_28_40_U1U_dz10=0.0.masks',
 'G:\\My Drive\\Projects\\MPLC\\results\\lab\\2024_10_08_find_z10\\finer\\2024_10_09_11_36_17_U1U_dz10=0.002.masks',
 'G:\\My Drive\\Projects\\MPLC\\results\\lab\\2024_10_08_find_z10\\finer\\2024_10_09_11_44_47_U1U_dz10=0.004.masks',
 'G:\\My Drive\\Projects\\MPLC\\results\\lab\\2024_10_08_find_z10\\finer\\2024_10_09_11_48_29_U1U_dz10=0.008.masks',
 'G:\\My Drive\\Projects\\MPLC\\results\\lab\\2024_10_08_find_z10\\finer\\2024_10_09_11_52_07_U1U_dz10=0.01.masks',
 'G:\\My Drive\\Projects\\MPLC\\results\\lab\\2024_10_08_find_z10\\finer\\2024_10_09_11_55_45_U1U_dz10=0.012.masks']

In [ ]:
# Signal scans - zaber motors are faster, and these are reflected. But maybe I should actually look at the detector that is not reflected, since I will not move it, so I want to find what is best for it, and then move the mirror accordingly... 
phases_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_19_14_12_fine_new_mirror_from_zero_more.phases"

for i in range(len(all_ds)):
    masks = get_good_masks(masks_path=masks_paths[i], modes_to_keep=modes_to_keep, phases_path=phases_path)
    mplc.load_masks(masks)
    mplc.restore_location()
    
    scan_saveto_path = rf'G:\My Drive\Projects\MPLC\results\lab\2024_10_08_find_z10\finer\{tnow()}_5spots_fine_signal_dz10={all_ds[i]}.scan'

    start_x = 12.65
    end_x = 13.2
    start_y = 5.6
    end_y = 7.05
    
    resolution = 1 
    x_pixels = 22
    y_pixels = 58
    pixel_size_x = 0.025 
    pixel_size_y = 0.025 
    
    scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path) 
    tt = get_timetagger(integration_time=1)
    t1 = run_in_thread(scanner.scan, x_motor=mxs, y_motor=mys, ph=tt)
    
    time.sleep(2)
    
    scan_saveto_path = rf'G:\My Drive\Projects\MPLC\results\lab\2024_10_08_find_z10\finer\{tnow()}_5spots_fine_idler_dz10={all_ds[i]}.scan'
    
    start_x = 9.0
    end_x = 9.6
    start_y = 3.4
    end_y = 4.9

    x_pixels = 24
    y_pixels = 60
    pixel_size_x = 0.025
    pixel_size_y = 0.025
    
    scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path)
    tt2 = get_timetagger(integration_time=1)
    t2 = run_in_thread(scanner.scan, x_motor=mxi, y_motor=myi, ph=tt2)
    t1.join()
    t2.join()    

God timetagger


In [24]:
mxs.move_absolute(12.94)
mys.move_absolute(5.84)
print(mxs.get_position())
print(mys.get_position())

12.93999825
5.840015625


In [6]:
backlash = 0
wait_after_move = 0.2
mxi, myi = get_idler_motors(backlash=backlash, wait_after_move=wait_after_move)

Got Thorlabs motors


In [29]:
# Idler scans - I don't want to move detectors, so the best z here will be the one I aim for also for the second detector by changing the mirror location / angle 
phases_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_19_14_12_fine_new_mirror_from_zero_more.phases"
for i in range(len(all_ds)):
    masks = get_good_masks(masks_path=masks_paths[i], modes_to_keep=modes_to_keep, phases_path=phases_path)
    mplc.load_masks(masks)
    mplc.restore_location()
    
    scan_saveto_path = rf'{dir_path}\{tnow()}_2spots_fine_idler_dz10={all_ds[i]}.scan'

    start_x = 9.1
    end_x = 9.6
    start_y = 4.1
    end_y = 4.9
    
    resolution = 1 
    x_pixels = 20
    y_pixels = 32
    pixel_size_x = 0.025 
    pixel_size_y = 0.025 
    
    scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path)   
    _ = scanner.scan(x_motor=mxi, y_motor=myi, ph=tt)

[0.         0.         5.21490009 0.         0.         0.
 0.         0.59253797 0.         0.         0.         0.
 1.41333115 0.         0.         0.         0.         1.02995202
 0.         0.         0.         0.         5.50020255 0.
 0.         0.         0.         0.33918276 0.         0.
 0.         0.         6.19378008 0.         0.         0.
 0.         0.10820319 0.         0.         0.         0.
 0.46709363 0.         0.         0.         0.         6.00202645
 0.         0.        ]
Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 4221. Singles2: 7778. Coincidence: 2.00.
dur: 2. pix: 0, 1. Singles1: 4428. Singles2: 7665. Coincidence: 0.00.
dur: 4. pix: 0, 2. Singles1: 4592. Singles2: 7829. Coincidence: 0.00.
dur: 5. pix: 0, 3. Singles1: 4878. Singles2: 7722. Coincidence: 0.00.
dur: 7. pix: 0, 4. Singles1: 4946. Singles2: 7679. Coincidence: 0.00.
dur: 9. pix: 0, 5. Singles1: 4971. Singles2: 7660. Coincidence: 0.00.
dur: 11. pix: 0, 6. Sin

In [30]:
# Finally, the really important thing to check is the correlation! 
# The signal is in a good location, and I will search for coincidences in the corresponding idlerignal spot (thorlabs are slower, but I will have a large integration time anyway...)
phases_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_19_14_12_fine_new_mirror_from_zero_more.phases"

integration_time = 10
coin_window = 2e-9
tt = get_timetagger(coin_window=coin_window, integration_time=integration_time)

for i in range(len(all_ds)):
    masks = get_good_masks(masks_path=masks_paths[i], modes_to_keep=modes_to_keep, phases_path=phases_path)
    mplc.load_masks(masks)
    mplc.restore_location()
    
    scan_saveto_path = rf'{dir_path}\{tnow()}_fine_corrs_dz10={all_ds[i]}.scan'

    start_x = 9.1
    end_x =   9.6
    start_y = 4.4
    end_y = 4.9
    
    resolution = 1 
    x_pixels = 20
    y_pixels = 20
    pixel_size_x = 0.025 
    pixel_size_y = 0.025 
    
    scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path)   
    _ = scanner.scan(x_motor=mxi, y_motor=myi, ph=tt)

God timetagger
[0.         0.         5.21490009 0.         0.         0.
 0.         0.59253797 0.         0.         0.         0.
 1.41333115 0.         0.         0.         0.         1.02995202
 0.         0.         0.         0.         5.50020255 0.
 0.         0.         0.         0.33918276 0.         0.
 0.         0.         6.19378008 0.         0.         0.
 0.         0.10820319 0.         0.         0.         0.
 0.46709363 0.         0.         0.         0.         6.00202645
 0.         0.        ]
Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 3856. Singles2: 7672. Coincidence: 0.10.
dur: 11. pix: 0, 1. Singles1: 3894. Singles2: 7605. Coincidence: 0.10.
dur: 22. pix: 0, 2. Singles1: 3971. Singles2: 7612. Coincidence: 0.30.
dur: 32. pix: 0, 3. Singles1: 4209. Singles2: 7627. Coincidence: 0.20.
dur: 43. pix: 0, 4. Singles1: 4527. Singles2: 7632. Coincidence: 0.30.
dur: 54. pix: 0, 5. Singles1: 5022. Singles2: 7659. Coincidence: 0.50.
dur

Traceback (most recent call last):
  File "C:\code\pianoq_results\pianoq_results\scan_result.py", line 184, in saveto
    f = open(path, 'wb')
    
FileNotFoundError: [Errno 2] No such file or directory: 'G:\\My Drive\\Projects\\MPLC\\results\\lab\\2024_10_08_find_z10\\2024_10_08_22_55_04_fine_corrs_dz10=-0.009.scan'


dur: 1380. pix: 6, 7. Singles1: 5074. Singles2: 10106. Coincidence: 0.00.
dur: 1391. pix: 6, 8. Singles1: 5148. Singles2: 10019. Coincidence: 0.10.
dur: 1402. pix: 6, 9. Singles1: 5379. Singles2: 10048. Coincidence: 0.20.
dur: 1412. pix: 6, 10. Singles1: 5551. Singles2: 10038. Coincidence: 0.70.
dur: 1423. pix: 6, 11. Singles1: 5604. Singles2: 10093. Coincidence: 1.00.
dur: 1434. pix: 6, 12. Singles1: 5472. Singles2: 10153. Coincidence: 0.70.
dur: 1445. pix: 6, 13. Singles1: 5066. Singles2: 10064. Coincidence: 1.10.
dur: 1455. pix: 6, 14. Singles1: 4497. Singles2: 10139. Coincidence: 0.40.
dur: 1466. pix: 6, 15. Singles1: 4136. Singles2: 10073. Coincidence: 0.50.
dur: 1477. pix: 6, 16. Singles1: 3903. Singles2: 10044. Coincidence: 0.40.
dur: 1488. pix: 6, 17. Singles1: 3722. Singles2: 10098. Coincidence: 0.10.
dur: 1498. pix: 6, 18. Singles1: 3624. Singles2: 10174. Coincidence: 0.20.
dur: 1509. pix: 6, 19. Singles1: 3588. Singles2: 10097. Coincidence: 0.20.
dur: 1522. pix: 7, 0. Single

# After finding dz10

In [31]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1 

masks_path =    fr'{dir_path}\{tnow()}_U1U_dz10=0.006.masks'
mplc_res_path = fr'{dir_path}\{tnow()}_U1U_dz10=0.006.mplcsim'
mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path, dead_middle_zone=0, last_plane_extra_dist=6e-3)
mplc_sim.res.saveto(mplc_res_path, smaller=True)

{'wavelength': 8.1e-07, 'dist_after_plane': array([0.087, 0.087, 0.087, 0.087, 0.138, 0.087, 0.087, 0.087, 0.087,
       0.093]), 'N_iterations': 15, 'Nx': 140, 'Ny': 360, 'dx': 1.25e-05, 'dy': 1.25e-05, 'max_k_constraint': 0.15, 'min_log_level': 2, 'size_factor': 3, 'use_mask_offset': True, 'symmetric_masks': False, 'active_planes': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'N_modes': 10}


100%|██████████| 15/15 [03:11<00:00, 12.79s/it]


In [34]:
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_10_08_find_z10\2024_10_09_10_20_53_U1U_dz10=0.006.masks"
phases_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_19_14_12_fine_new_mirror_from_zero_more.phases"

masks = get_good_masks(masks_path=masks_paths[i], modes_to_keep=modes_to_keep, phases_path=phases_path)
mplc.load_masks(masks)

In [35]:
mxi.close()
myi.close()
mxs.close()
mys.close()